In [24]:
%%HTML
<h1>Introduction</h1>
<h3>Trading Strategy Backtesting with Python</h3>
<p>This is a a simple Intraday Trading Strategy Algorithm, which uses <a href="https://pypi.org/project/alpha-vantage/"> alpha_vantage API module </a> of <a href="https://www.alphavantage.co/"> Alpha Vantage</a>. Here all the data is fetched using Timeseries method developed by Alpha Vantage</p>
<p>
In this Jupyter notebook, we will explore how to build and backtest a simple trading strategy using Python. The strategy will be based on a simple technical analysis indicator, and we will use the Alpha Vantage API to fetch historical intraday data for a given stock.

We will first define the necessary classes and functions to fetch and preprocess the data, generate buy/sell signals based on our strategy, and plot a candlestick chart with our strategy overlaid on top. Then, we will run the strategy for a specific stock and examine the resulting signals and chart.

This notebook assumes some prior knowledge of Python and financial markets. Let's get started! </p>

In [25]:
%%HTML
<h1>Import Statements </h1>

In [26]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go
from dotenv import load_dotenv
import os

In [27]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [28]:
%%HTML
<h1> Class ScriptData </h1>
<p>This Class to store and manage the intraday data of a script.</p>
<h3> Method fetch intraday data </h3>
<p>This method fetches the intraday data of the given script from the API</p>
<h3> Method convert intraday data </h3>
<p>This method converts the fetched intraday data of the given script in proper usable form and stores it in Dataframe.</p>

In [29]:

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.intraday_data = {}

    def fetch_intraday_data(self, script):
        ts = TimeSeries(key=self.api_key, output_format='pandas')
        self.intraday_data[script], meta_data = ts.get_intraday(symbol=script, interval='60min', outputsize='full')

    def convert_intraday_data(self, script):
        data = self.intraday_data[script]
        data.reset_index(inplace=True)
        data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data[['open', 'high', 'low', 'close', 'volume']] = data[['open', 'high', 'low', 'close', 'volume']].astype(
            float)
        data['volume'] = data['volume'].astype(int)
        self.intraday_data[script] = data

    def __getitem__(self, script):
        return self.intraday_data[script]

    def __setitem__(self, script, value):
        self.intraday_data[script] = value

    def __contains__(self, script):
        return script in self.intraday_data

In [30]:
%%HTML
<h1> Function Indicator </h1>
<p>Calculates the simple moving average of the close price and stores the timestamp and indicator only in DF.</p>

In [31]:
def indicator1(df, timeperiod=15):
    ma = df['close'].rolling(window=timeperiod).mean()
    result = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': ma})
    return result


In [32]:
%%HTML
<h1> Class Strategy </h1>
<p>This Class to implement the trading strategy.</p>

<h1> Method Fetch Data </h1>
<p>This method fetches the intraday data of the given script from the API, converts the fetched intraday data of the given script in proper usable form and stores it in Dataframe and finally uses the indicator function to return and store the usable data only.</p>

<h1> Method Generate Signals </h1>
<p>This method uses the data returned by above method to generate Buy/Sell signals and refactors the complete data into only the necessary data.</p>

<h1> Method Plot Candelstick Chart </h1>
<p>This method takes the data returned from above methods and the complete data and compare both then plot a candlestick chart of them.</p>

<h1> Method Run </h1>
<p>This method is created so you dont have to call all the methods one by one. You just need to call the strategy class and then call the run method.</p>


In [33]:

class Strategy:
    def __init__(self, script):
        self.script = script
        self.script_data = ScriptData(api_key)

    def fetch_data(self):
        self.script_data.fetch_intraday_data(self.script)
        self.script_data.convert_intraday_data(self.script)
        self.df = self.script_data[self.script]
        self.indicator_data = indicator1(self.df)
        self.close_data = self.df['close']
        self.indicator_data = indicator1(self.df)

    def generate_signals(self):
        self.fetch_data()
        self.indicator_data['signal'] = np.where((self.indicator_data['indicator'] > self.close_data) & (
                self.indicator_data['indicator'].shift(1) <= self.close_data.shift(1)), 'BUY',
                                                 np.where((self.indicator_data['indicator'] < self.close_data) & (
                                                         self.indicator_data['indicator'].shift(
                                                             1) >= self.close_data.shift(1)),
                                                          'SELL', 'NO_SIGNAL'))

        dropind = self.indicator_data[self.indicator_data['signal'] == 'NO_SIGNAL'].index
        self.indicator_data.drop(dropind, inplace=True)
        self.df.drop(dropind, inplace=True)

        self.signals = self.indicator_data[
            (self.indicator_data['signal'] == 'BUY') | (self.indicator_data['signal'] == 'SELL')]
        self.signals.reset_index(inplace=True)
        self.signals = self.signals.drop(columns=['index', 'indicator'])
        
    def plot_candlestick_chart(self, df, indicator_data):
        fig = go.Figure(
            data=[
                go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close'])])
        fig.add_trace(go.Scatter(x=indicator_data['timestamp'], y=indicator_data['indicator'], line=dict(color='grey')))
        fig.show()

    def run(self):
        self.generate_signals()
        print(self.signals)
        self.plot_candlestick_chart(self.df, self.indicator_data)

In [34]:
%%HTML
<h1> Example </h1>
<p>Here is an example, just call the strategy class with your script as an input and then call the run method on the class. Thats it.</p>

In [35]:
strategy = Strategy("MSFT")
strategy.run()

             timestamp signal
0  2023-02-13 19:00:00    BUY
1  2023-02-13 14:00:00   SELL
2  2023-02-13 11:00:00    BUY
3  2023-02-13 10:00:00   SELL
4  2023-02-13 09:00:00    BUY
..                 ...    ...
60 2022-12-22 15:00:00    BUY
61 2022-12-22 10:00:00   SELL
62 2022-12-21 12:00:00    BUY
63 2022-12-21 11:00:00   SELL
64 2022-12-21 10:00:00    BUY

[65 rows x 2 columns]
